In [ ]:
# carregar bibliotecas

from samgeo.text_sam import LangSAM
import glob
import os
import pandas as pd
from PIL import Image
import numpy as np

In [ ]:
# escolha do modelo
# vit_b(91M), vit_l(308M), vit_h(636M)

sam = LangSAM(model_type='vit_b')

In [ ]:
import os
import glob
from PIL import Image
import numpy as np

def processar_e_salvar_maior_mascara(caminho_pasta, formato, texto_prompt, box_t, text_t, pasta_saida):
    """
    Processa as imagens de uma pasta, seleciona e salva o recorte da maior máscara.

    Args:
      caminho_pasta: Caminho da pasta que contém as imagens.
      formato: Formato das imagens de entrada.
      texto_prompt: Prompt de texto para a predição.
      box_t: Threshold para a detecção de caixas delimitadoras.
      text_t: Threshold para a detecção de texto.
      pasta_saida: Pasta de saída dos arquivos.

    Returns:
      Nenhum.
    """

    imagens = glob.glob(f"{caminho_pasta}/*.{formato}")
    os.makedirs(pasta_saida, exist_ok=True)

    for imagem in imagens:
        print(f"Processando imagem: {imagem}")

        # Processar imagem e obter máscara
        resultado = sam.predict(image=imagem,
                                text_prompt=texto_prompt,
                                box_threshold=box_t,
                                text_threshold=text_t)

        if resultado is None:
            print(f"Nenhuma máscara encontrada em: {imagem}")
            continue

        # Identificar a maior máscara
        maior_mascara = max(resultado, key=lambda x: np.sum(x['mask']))

        # Carregar imagem original
        img_original = Image.open(imagem)

        # Recortar a imagem com base na maior máscara
        bbox = maior_mascara['bbox']  # bbox é uma tupla (x_min, y_min, x_max, y_max)
        recorte = img_original.crop(bbox)

        # Salvar recorte
        caminho_saida = f"{pasta_saida}/{os.path.basename(imagem)}_{texto_prompt}.png"
        recorte.save(caminho_saida)

        print(f"Recorte salvo: {caminho_saida}")


In [ ]:
# Uso

# caminho_pasta = 'G:\\drone_img_sam\\praia_brava_16-12-2023\\101MEDIA_georref\\'
caminho_pasta = '/Users/jac/Documents/temp/imgs_drone_praia_brava/101_georref'
formato = 'png'
texto_prompt = 'car'
box_t = 0.19
text_t = 0.19
pasta_saida = 'saida/processamento_lote/pessoas0.19'

processar_imagens(caminho_pasta, formato, texto_prompt,
                  box_t, text_t, pasta_saida)

unir_shapefiles(pasta_saida)